# OMERO image annotation Workflow with micro-sam annotation plugin


## 1. Setup and Installation

In [1]:
# Import the streamlined package
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
)

# System imports
import os
from pathlib import Path

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
# Check dependencies
try:
    import ezomero
    print(f"🔗 OMERO functionality: ✅ Available")
except ImportError:
    print(f"🔗 OMERO functionality: ❌ Install with: pip install -e .[omero]")



📦 omero-annotate-ai version: 0.1.0
🔗 OMERO functionality: ✅ Available
🔑 Keyring support: ✅ Available


## 2. OMERO Connection

In [ ]:
# Create and display the OMERO connection widget
print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server:")
print("  • Fill in server details (host, username, password)")
print("  • Test connection before proceeding")
print("  • Choose password storage duration if desired")
print("  • Click 'Save & Connect' to establish connection")
print()

conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Next Step: Run the cell below after connecting to OMERO")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server:
  • Fill in server details (host, username, password)
  • Test connection before proceeding
  • Choose password storage duration if desired
  • Click 'Save & Connect' to establish connection

Loaded configuration from connection history: root@localhost (users)
Password loaded from keychain (no expiration)



📝 Next Step: Run the cell below after connecting to OMERO


In [4]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection established!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
print(f"🔐 Secure: {conn.isSecure()}")
print(f"🔗 Connection ready for workflow setup")

✅ OMERO connection established!
👤 User: root
🏢 Group: users
🔐 Secure: True
🔗 Connection ready for workflow setup


In [5]:
from omero_annotate_ai.omero.omero_utils import get_container_info

get_container_info(conn,"project",101)


## 3. Workflow Configuration

In [6]:
# Auto-reload package edits in this notebook
%load_ext autoreload
%autoreload 2

In [7]:
# Create and display the workflow widget
print("🔬 OMERO Annotation Workflow Setup")
print("Follow the sequential workflow below:")
print("  1. Select working directory")
print("  2. Choose OMERO container")
print("  3. Check existing annotation tables")
print("  4. Configure micro-SAM parameters")
print("  5. Save configuration")
print()

workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

print("\n📝 Next Step: Complete the workflow above, then run the cell below")

🔬 OMERO Annotation Workflow Setup
Follow the sequential workflow below:
  1. Select working directory
  2. Choose OMERO container
  3. Check existing annotation tables
  4. Configure micro-SAM parameters
  5. Save configuration




📝 Next Step: Complete the workflow above, then run the cell below


In [8]:
# Helper to reload package and rebuild the widget without resetting the notebook
import importlib, sys
import omero_annotate_ai as _oaa
from omero_annotate_ai import create_workflow_widget as _mk_widget
from omero_annotate_ai.omero.omero_utils import get_container_info as _probe  # sanity import
def reload_widget(existing_conn=None):
    # Reload top-level package and commonly edited submodules
    importlib.reload(_oaa)
    # Optionally reload specific modules if needed (safe if they exist)
    for modname in list(sys.modules):
        if modname.startswith('omero_annotate_ai.'):
            try:
                importlib.reload(sys.modules[modname])
            except Exception:
                pass
    # Re-import factory to ensure we have the latest symbol
    from omero_annotate_ai import create_workflow_widget as _mk_widget2
    w = _mk_widget2(connection=existing_conn)
    display(w.main_widget)
    return w

# Usage:
workflow_widget = reload_widget(existing_conn=conn)

## 4. Pipeline Execution


#### 4.1. Configuration summary

In [9]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

# Display configuration summary
print("\n📋 Configuration Summary:")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🎯 Training Set: {config.name}")
print(f"   🔬 Model: {config.ai_model.model_type}")
print(f"   📺 Channel: {config.spatial_coverage.channels}")
print(f"   📁 Output: {config.output.output_directory}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"   🧩 Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})"
    )

if config.spatial_coverage.three_d:
    print(f"   🧊 3D processing: Enabled")


print(f"\n📊 Processing: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")


📋 Configuration Summary:
   📦 Container: dataset (ID: 0)
   🎯 Training Set: default_annotation_workflow
   🔬 Model: vit_b_lm
   📺 Channel: [0]
   📁 Output: annotations
   🔄 Resume from Table: False
   📖 Read-only Mode: False

📊 Processing: 3 training + 3 validation


#### 4.2. Convert configuration to an annotation pipeline

In [ ]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"🔍 Validating {container_type} with ID {container_id}...")

# Validate container exists
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")

print(f"✅ Found {container_type}: {container.getName()}")
if container.getDescription():
    print(f"📝 Description: {container.getDescription()}")

#### 4.3. Run the annotation pipeline on the files
After loading of the files the napari will open for you to annotate the images

In [ ]:
# Run the complete annotation pipeline
print("🚀 Starting annotation pipeline...")
print(f"   🔬 Using micro-SAM model: {config.ai_model.model_type}")

if config.processing.batch_size == 0:
    print(f"   ⚡ Processing: All images in one batch")
else:
    print(f"   📦 Processing: Batches of {config.processing.batch_size} images")

print(f"   🎨 Napari will open for interactive annotation")
print(f"   📝 Close napari windows when annotation is complete")

try:
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n🎉 Annotation pipeline completed successfully!")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.output.output_directory}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

#### Save the configuration file to the local folder

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"💾 Configuration saved to: {config_path}")
except Exception as e:
    print(f"⚠️ Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"💾 Configuration saved to: {config_filename}")

print(f"\n📋 Configuration Summary:")
print(f"   Name: {config.name}")
print(f"   Description: {config.description}")
print(f"   Output Directory: {config.output.output_directory}")
print(f"   Model Type: {config.ai_model.model_type}")

print(f"\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

## 6. Cleanup

Close the OMERO connection when finished.

In [ ]:
# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print(f"\n🎉 Streamlined workflow completed!")
print(f"📊 Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"💾 Configuration saved for future use")
print(f"✨ Ready for next annotation workflow!")